In [52]:
import mne 
import numpy as np
import pandas as pd

# parameters
n_samples_per_midiepoch = 900
n_samples_ini = 20
n_midiepochs_per_trial = 4


# load data
bids_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids'
subject = 'S04'
fn_movie = f'sub-{subject}_Dynamic_preprocessed-epo.fif'
epochs_movie = mne.read_epochs(f'{bids_dir}/derivatives/preprocessed/{fn_movie}')
epochs_movie=epochs_movie[epochs_movie.metadata['condition']!='catch']

# initialize
n_trials = epochs_movie._data.shape[0]
n_channels = epochs_movie._data.shape[1]
n_timepoints = epochs_movie._data.shape[2]


Reading /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/sub-S04_Dynamic_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    3200.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
488 matching events found
No baseline correction applied
0 projection items activated


In [53]:
onset_first_miniepoch = np.where(epochs_movie.times==0)[0][0]+n_samples_ini

# setup for metadata
miniepochs = []
condition = epochs_movie.metadata['condition']
direction = [c.split('_')[1] for c in condition]
right_idx = [d=='Right' for d in direction]
left_idx = [d=='Left' for d in direction]
degree = [int(c.split('_')[0]) for c in condition] 
run = epochs_movie.metadata['run_nr']

for i in range(n_midiepochs_per_trial):
    # onset shifts by 20 samples, going over the entire movie
    # remember the first mini epoch is not at image onset but 20 samples later
    onset = onset_first_miniepoch+i*n_samples_per_midiepoch
    me = mne.EpochsArray(epochs_movie._data[:,:,onset:onset+n_samples_per_midiepoch],info=epochs_movie.info)

    # work out the wrap-around in circular space (can't have numbers >360 or <0)
    deg = np.zeros(len(degree))
    deg[right_idx] = np.array(degree)[right_idx]+45*i
    deg[left_idx] = np.array(degree)[left_idx]-45*i

    deg[deg>359] = deg[deg>359]-360
    deg[deg<0] = 359-np.abs(deg[deg<0])

    # make pandas metadata
    df = pd.DataFrame()
    df['condition']=condition
    df['direction']=direction
    df['start_pos']=deg
    df['end_pos']=deg + 45
    df['chunk']=i
    df['trialnum']=np.arange(n_trials)
    df['run'] = run

    me.metadata = df
    miniepochs.append(me)


Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 7 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 7 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 7 columns
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 7 columns


In [31]:
print(miniepochs[0][1])

<EpochsArray |  1 events (all good), 0 – 0.749167 s, baseline off, ~2.1 MB, data loaded, with metadata,
 '1': 1>


In [54]:
# concatenate all timepoints
mini_epochs = mne.concatenate_epochs(miniepochs)

# save them
file_name = f'{bids_dir}/derivatives/preprocessed/sub-{subject}_midiEpochs_preprocessed-epo.fif'
mini_epochs.save(file_name, overwrite=True)

Adding metadata with 7 columns
1792 matching events found
No baseline correction applied
